# Logistic Regression

In [ ]:
%pip install \
    git+https://github.com/deepmind/dm-haiku@v0.0.4 \
    git+https://github.com/deepmind/optax@v0.0.9

In [2]:
import haiku as hk

from jax import jit, partial, vmap, grad
from jax import random
import jax.lax as lax
import jax.nn as nn
import jax.numpy as np
from jax.scipy.special import logit

import optax

In [3]:
@hk.without_apply_rng
@hk.transform
def model(x):
    linear = hk.Linear(output_size=1)
    return linear(x)

In [4]:
rng = random.PRNGKey(42)

In [5]:
x_shape = (10,)
rng, r = random.split(rng)
generating_model_state = model.init(r, np.zeros(x_shape))
rng, r = random.split(rng)
x = random.normal(r, (1000,) + x_shape)
y = vmap(partial(model.apply, generating_model_state))(x)
y = np.array(y >= 0., float)

@jit
def loss_fn(model_state):
    model_predictions_logits = vmap(partial(model.apply, model_state))(x)
    loss = optax.sigmoid_binary_cross_entropy(model_predictions_logits, y)
    loss = np.mean(loss)
    return loss

@jit
def accuracy(model_state):
    model_predictions_logits = vmap(partial(model.apply, model_state))(x)
    yp = np.array(model_predictions_logits >= 0., float)
    return np.mean(y == yp)

In [6]:
steps = 50
start_learning_rate = 1e-2
optimizer = optax.adamw(start_learning_rate, weight_decay=1e-2)

@jit
def train(model_state, optimizer_state):
    def train_step(i, train_state):
        model_state, optimizer_state = train_state
        loss_grads = grad(loss_fn)(model_state)
        model_updates, optimizer_state = optimizer.update(loss_grads, optimizer_state, model_state)
        model_state = optax.apply_updates(model_state, model_updates)
        return model_state, optimizer_state

    initial_train_state = model_state, optimizer_state
    return lax.fori_loop(0, steps, train_step, initial_train_state)

In [7]:
rng, r = random.split(rng)
inferred_model_state = model.init(r, np.zeros(x.shape[1:]))
optimizer_state = optimizer.init(inferred_model_state)

In [8]:
for i in range(10):
    print(accuracy(inferred_model_state))
    inferred_model_state, optimizer_state = train(inferred_model_state, optimizer_state)
print(accuracy(inferred_model_state))

0.518
0.77900004
0.87700003
0.91700006
0.94200003
0.95500004
0.961
0.97
0.97200006
0.9760001
0.97800004
